# Internet and Websites Search using Bing API - Bing Chat Clone

In this notebook, we'll delve into the ways in which you can **boost your GPT Smart Search Engine with web search functionalities**, utilizing both Langchain and the Azure Bing Search API service.

As previously discussed in our other notebooks, **harnessing agents and tools is an effective approach**. We aim to leverage the capabilities of OpenAI's large language models (LLM), such as GPT-4 and its successors, to perform the heavy lifting of reasoning and researching on our behalf.

There are numerous instances where it is necessary for our Smart Search Engine to have internet access. For instance, we may wish to **enrich an answer with information available on the web**, or **provide users with up-to-date and recent information**, or **finding information on an specific public website**. Regardless of the scenario, we require our engine to base its responses on search results.

By the conclusion of this notebook, you'll have a solid understanding of the Bing Search API basics, including **how to create a Web Search Agent using the Bing Search API**, and how these tools can strengthen your chatbot. Additionally, you'll learn about **Callback Handlers, their use, and their significance in bot applications**.

In [1]:
import requests
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.utilities import BingSearchAPIWrapper

from common.callbacks import StdOutCallbackHandler
from common.prompts import BING_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string.replace("$","USD ")))

# GPT-4 models are necessary for this feature. GPT-35-turbo will make mistakes multiple times on following system prompt instructions.
MODEL_DEPLOYMENT_NAME = "gpt-4-32k" 

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Introduction to Callback Handlers

This following explanation comes directly from the Langchain documentation about Callbacks ([HERE](https://python.langchain.com/docs/modules/callbacks/)):

**Callbacks**:<br>
LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks. You can subscribe to these events by using the callbacks argument available throughout the API. This argument is list of handler objects.

**Callback handlers**:<br>
CallbackHandlers are objects that implement the CallbackHandler interface, which has a method for each event that can be subscribed to. The CallbackManager will call the appropriate method on each handler when the event is triggered.

--------------------
We will incorporate a handler for the callbacks, enabling us to observe the response as it streams and to gain insights into the Agent's reasoning process. This will prove incredibly valuable when we aim to stream the bot's responses to users and keep them informed about the ongoing process as they await the answer.

Our custom handler is on the folder `common/callbacks.py`. Go and take a look at it.

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

# Now we declare our LLM object with the callback handler 
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000)

# or uncomment the below line if you want to see the responses being streamed
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000, streaming=True, callback_manager=cb_manager)

## Creating a custom tool - Bing Search API tool

Langhain has already a pre-created tool called BingSearchAPIWrapper ([HERE](https://github.com/hwchase17/langchain/blob/master/langchain/utilities/bing_search.py)), however we are going to make it a bit better by using the results function instead of the run function, that way we not only have the text results, but also the title and link(source) of each snippet.

In [4]:
class MyBingSearch(BaseTool):
    """Tool for a Bing Search Wrapper"""
    
    name = "@bing"
    description = "useful when the questions includes the term: @bing.\n"

    k: int = 5
    
    def _run(self, query: str) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        return bing.results(query,num_results=self.k)
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

Now, we **create our REACT agent that uses our custom tool and our custom promp**t `BING_PROMPT_PREFIX`

In [5]:
tools = [MyBingSearch(k=5)] # assigning custom toll created above
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         agent_kwargs={'prefix':BING_PROMPT_PREFIX}, callback_manager=cb_manager, )

Try some of the below questions, or others that you might like

In [6]:
# QUESTION = "Create a list with the main facts on What is happening with the oil supply in the world right now?"
# QUESTION = "How much is 50 USD in Euros and is it enough for an average hotel in Madrid?"
# QUESTION = "My son needs to build a pinewood car for a pinewood derbi, how do I build such a car?"
# QUESTION = "Who won the 2023 superbowl and who was the MVP?"
# QUESTION = "can I travel to Hawaii, Maui from Dallas, TX for 7 days with $7000 on the month of September, what are the best days to travel?"


# This complex question below needs gpt-4-32k (0613 version) in order to ensure a good answer. 
# ---------------
QUESTION = """
compare the number of job opennings (provide the exact number), the average salary within 15 miles of Dallas, TX, for these ocupations:

- ADN Registerd Nurse 
- Occupational therapist assistant
- Dental Hygienist
- Graphic Designer
- Real Estate Agent


Create a table with your findings. Place the sources on each cell.
"""

In [7]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

The user is asking for a comparison of job openings and average salaries for five different occupations within a 15-mile radius of Dallas, TX. I will need to perform multiple searches to gather this information. I will start by searching for the number of job openings and average salary for an ADN Registered Nurse in Dallas, TX. 

Action: @bing
Action Input: ADN Registered Nurse job openings and average salary within 15 miles of Dallas, TXThe user is asking for a comparison of job openings and average salaries for five different occupations within a 15-mile radius of Dallas, TX. I will need to perform multiple searches to gather this information. I will start by searching for the number of job openings and average salary for an ADN Registered Nurse in Dallas, TX. 

Action: @bing
Action Input: ADN Registered Nurse job openings and average salary within 15 miles of Dallas, TX
The search results show that there are varying numbers of job openings for ADN Registered Nurses in Dallas, TX, w

In [8]:
printmd(response)

Here is the comparison of job openings and average salaries for the specified occupations within a 15-mile radius of Dallas, TX:

| Occupation | Job Openings | Average Salary | Sources |
|------------|--------------|----------------|---------|
| ADN Registered Nurse | 680-1,048 | USD 35.52 per hour | <sup><a href="https://www.indeed.com/q-RN-Adn-l-Dallas,-TX-jobs.html" target="_blank">[1]</a></sup> <sup><a href="https://www.ziprecruiter.com/Salaries/ADN-Nurse-Salary-in-Dallas,TX" target="_blank">[2]</a></sup> |
| Occupational Therapist Assistant | Not specified | USD 78,659 per year | <sup><a href="https://www.glassdoor.com/Salaries/dallas-occupational-therapy-assistant-salary-SRCH_IL.0,6_IM218_KO7,37.htm" target="_blank">[3]</a></sup> |
| Dental Hygienist | 154 | USD 48.69 per hour | <sup><a href="https://www.indeed.com/career/dental-hygienist/salaries/Dallas--TX" target="_blank">[4]</a></sup> <sup><a href="https://www.indeed.com/q-Dental-Hygienist-l-Dallas,-TX-jobs.html" target="_blank">[5]</a></sup> |
| Graphic Designer | 87 | USD 21.92 per hour | <sup><a href="https://www.indeed.com/career/graphic-designer/salaries/Dallas--TX" target="_blank">[6]</a></sup> <sup><a href="https://www.indeed.com/q-Graphics-Designer-l-Dallas,-TX-jobs.html" target="_blank">[7]</a></sup> |
| Real Estate Agent | 467 | USD 98,544 per year | <sup><a href="https://www.indeed.com/career/real-estate-agent/salaries/Dallas--TX" target="_blank">[8]</a></sup> <sup><a href="https://www.indeed.com/q-Real-Estate-Agent-l-Dallas,-TX-jobs.html" target="_blank">[9]</a></sup> |

Please note that the number of job openings and average salaries can vary and are subject to change. It's always a good idea to check multiple sources for the most accurate and up-to-date information.

## QnA to specific websites

There are several use cases where we want the smart bot to answer questions about a specific company's public website. There are two approaches we can take:

1. Create a crawler script that runs regularly, finds every page on the website, and pushes the documents to Azure Cognitive Search.
2. Since Bing has likely already indexed the public website, we can utilize Bing search targeted specifically to that site, rather than attempting to index the site ourselves and duplicate the work already done by Bing's crawler.

Below are some sample questions related to specific sites. Take a look:

In [9]:
# QUESTION = "information on how to kill wasps in homedepot.com"
# QUESTION = "in target.com, find how what's the price of a Nesspresso coffee machine and of a Keurig coffee machine"
# QUESTION = "in microsoft.com, find out what is the latests news on quantum computing"
QUESTION = "give me on a list the main points on the latest investor report from mondelezinternational.com"

In [10]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(3):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

The user is asking for the main points from the latest investor report from Mondelez International. I will need to search for this specific report on the Mondelez International website.
Action: @bing
Action Input: latest investor report site:mondelezinternational.comThe user is asking for the main points from the latest investor report from Mondelez International. I will need to search for this specific report on the Mondelez International website.
Action: @bing
Action Input: latest investor report site:mondelezinternational.com
The search results provide some information about the latest investor report from Mondelez International. The most relevant information seems to be in the third and fourth snippets. However, to ensure I provide the most accurate and comprehensive information, I should perform a second search on the specific page that seems to contain the latest report.
Action: @bing
Action Input: site:https://ir.mondelezinternational.com/news-releases/news-release-details/monde

In [11]:
printmd(response)

Here are the main points from the latest investor report (Q4 and FY 2021) from Mondelez International:

1. Net revenues increased by 8.0%<sup><a href="https://ir.mondelezinternational.com/news-releases/news-release-details/mondelez-international-reports-q4-and-fy-2021-results" target="_blank">[1]</a></sup>.
2. This increase was driven by Organic Net Revenue growth of 5.2%, favorable currency, and incremental sales from the company's acquisitions of Give & Go, Hu, Grenade, and Gourmet Food<sup><a href="https://ir.mondelezinternational.com/news-releases/news-release-details/mondelez-international-reports-q4-and-fy-2021-results" target="_blank">[1]</a></sup>.
3. Volume and pricing drove Organic Net Revenue growth<sup><a href="https://ir.mondelezinternational.com/news-releases/news-release-details/mondelez-international-reports-q4-and-fy-2021-results" target="_blank">[1]</a></sup>.
4. Diluted EPS was USD 3.04, up 23.1%<sup><a href="https://ir.mondelezinternational.com/static-files/e6dc02e4-63d1-4eb5-99ef-8ba229bc11b3" target="_blank">[2]</a></sup>.
5. Adjusted EPS was USD 2.87, up 9.0% on a constant-currency basis<sup><a href="https://ir.mondelezinternational.com/static-files/e6dc02e4-63d1-4eb5-99ef-8ba229bc11b3" target="_blank">[2]</a></sup>.

Is there anything else you would like to know?

In [12]:
# Uncomment if you want to take a look at the custom bing search prompt (This is where the magic happens: a great system promp + GPT-4)
# printmd(agent_executor.agent.llm_chain.prompt.template)

# Summary

In this notebook, we learned about Callback Handlers and how to stream the response from the LLM. We also learn how to create a Bing Chat clone using a clever prompt with specific search and formatting instructions.

The outcome is an agent capable of conducting intelligent web searches and performing research on our behalf. This agent provides us with answers to our questions along with appropriate URL citations and links!

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of all notebooks and create a brain agent that can respond to any request accordingly.